# Tag 2 - LLM Fine-Tuning

## Einführung in das Hugging Face Ökosystem

Um die Bestandteile des ML-Workflows und die technischen Details hinter ML-Modellen detaillierter zu beleuchten, haben wir gestern pures PyTorch verwendet, um unsere Daten zu laden und aufzubereiten, und unsere Modelle von Grund auf zu bauen, trainieren, und evaluieren. Für die meisten Aufgabenstellungen ist es heute jedoch üblich, bereits vortrainierte Modelle entweder direkt ohne Anpassungen zu verwenden (auch als *zero-shot* bezeichnet), oder diese für die eigene Problemstellung zu finetunen. 

Zu diesem Zweck hat sich [Hugging Face](https://huggingface.co/) sowohl in der Wissenschaft als auch im kommerziellen Bereich als die primäre Platform herauskristallisiert, um ML-Modelle, Datensätze, etc., auszutauschen. Zudem stellt Hugging Face eine weite Auswahl an Libraries zur Verfügung, die das Verwenden von bestehenden Modellen und Datensätzen, aber auch Aspekte wie Fine-Tuning, Optimierung für die Nutzung in Production, und Training auf Clustern oder spezieller Hardware (z.B. Google TPUs, Habana Gaudi Prozessoren) vereinfacht. 

## Hugging Face Transformers
Die wichtigste Hugging Face Library ist mit Abstand [Transformers](https://huggingface.co/docs/transformers/index). Sie kapselt die Nutzung von vortrainierten Modellen aus verschiedensten Einsatzgebieten, das für sie notwendige Pre- und Post-Processing, sowie ihr Training / Fine-Tuning.  

### Basics
Zur Verwendung eines Modells aus dem Hugging Face Ökosystem benötigt man immer mindestens 2 Komponenten: die richtige Model-Klasse, und die Pre-Processors, die die Inputs für die aktuelle Aufgabenstellung (Bilder, Text, etc.) in das Format umwandeln, dass das Modell erwartet. Laden wir ein Beispiel-Modell, das Language Model RoBERTa (https://huggingface.co/roberta-base):

In [52]:
from transformers import AutoTokenizer, RobertaForMaskedLM

roberta_mlm = RobertaForMaskedLM.from_pretrained("roberta-base")
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Die vorangehenden Zeilen laden das RoBERTa Modell, und seinen Pre-Processor, den Tokenizer: Da ein Language Model (grob gesagt) numerischen Input erwartet, muss ein Eingabe-String zuerst in kleinere Teile aufgeteilt werden, die *Tokens*. Je nach Modell kommen hier unterschiedliche Techniken zum Einsatz, bei denen ein Token immer ein gesamtes Wort, oder häufiger den Teil eines Wortes repräsentiert (Sub-Word Tokenization). Die Verwendung der `from_pretrained` Methode mit dem selben Model-Identifier stellt sicher, dass wir die richtige Kombination and Modell und Tokenizer laden.

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Der Tokenizer von RoBERTa ist ein Byte Pair Encoding (BPE) Tokenizer. Diese Technik wurde erstmals von OpenAI GPT-2 verwendet, und wird hier gut erläutert: https://lucytalksdata.com/the-modern-tokenization-stack-for-nlp-byte-pair-encoding/. 

<span style="color:white; background-color: blue; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Info: </span> Um von der Transformer-Architektur, auf der fast alle modernen Language Models basieren, verwendet zu werden, müssen die Tokens (einfache Ids) noch weiter in multi-dimensionale Vektoren umgewandelt werden. Dieser Prozess wird als *Embedding* bezeichnet. Der Prozess von Tokenization zum Embedding wird in diesem Artikel sehr umfangreich erklärt: https://www.lesswrong.com/posts/pHPmMGEMYefk9jLeh/llm-basics-embedding-spaces-transformer-token-vectors-are. 

Wie verwenden wir nun unser Modell? Wie du aus dem Code oben erkennen kannst, haben wir eine Variante von RoBERTa geladen, die für Masked Language Modelling (MLM) finetuned wurde, also das Füllen von Lücken in einem Text. Sehen wir uns ein Beispiel an:

In [53]:
import torch 

inputs = roberta_tokenizer("The capital of France is <mask>.", return_tensors="pt")

with torch.no_grad():
    logits = roberta_mlm(**inputs).logits

mask_token_index = (inputs.input_ids == roberta_tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
roberta_tokenizer.decode(predicted_token_id)

' Paris'

Wie erwartet leiten wir unseren Input-Satz durch den Tokenizer, und geben das Ergebnis des Tokenizers 1-zu-1 and unser Modell weiter. Das Modell gibt für den maskierten Token (in diesem Fall enthält der Satz nur einen) Gewichte zurück (sogenannte Logits), die ausdrücken, welchen Token das Modell an dieser Stelle am wahrscheinlichsten einsetzen würde. Wie erwartet füllt das Modell die Lücke im Text mit der korrekten Antwort: Paris. 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Sieh dir die `inputs` an das Modell genauer an. Stimmt die Anzahl an Tokens (`input_ids`) mit der Anzahl an Wörtern überein, und wenn nicht, was könnte die Ursache sein (um eine Vermutung zu entkräftigen, sieh dir den Output von `inputs.word_ids(batch_index=0)` an)? Was könnte der Zweck der `attention_mask` sein? 

<span style="color:white; background-color: #FFD700; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Bonus-Aufgabe: </span> Leite dir ab, wie der Code oben genau funktioniert, um an die Antwort des Modells zu kommen. Tipps: Das Modell verarbeitet gewöhnlich nicht nur einen, sondern einen Batch an Sätzen. `tokenizer.mask_token_id` enthält die ID des Tokens, in den `<mask>` umgewandelt wird (also die ID, an der wir interessiert sind). Bei guten Verständnis solltest du den Code so umbauen können, dass du die Antworten für einen Batch an Sätzen auf einmal auslesen kannst. 

<span style="color:white; background-color: red; padding: 3px 6px; border-radius: 2px; margin-right: 5px;">Aufgabe: </span> Transformers stellt sogenannte *Pipelines* zur Verfügung, die das Post-Processing der Model-Outputs übernehmen. Nutze die Pipelines API-Doc (https://huggingface.co/docs/transformers/v4.30.0/en/main_classes/pipelines) um das gleiche Ergebnis zu erhalten. Übergib der Pipeline dabei die bereits initialisierten Modell & Tokenizer.

In [ ]:
from transformers import pipeline

# Your code goes here

### Weitere Details zum Laden von Modellen

Wie bereits aus dem `with torch.no_grad()` Befehl erkennbar sein sollte, ist das mittels Transformers geladene Modell ein reguläres PyTorch Modell. Wir können uns daher die Details des Modells wie gewohnt ansehen, und auch sonst ohne die Helfer von Transformers mit dem Modell so interagieren, wie wir es von einem PyTorch Modell gewohnt sind:

In [54]:
from torchinfo import summary

summary(roberta_mlm)

Layer (type:depth-idx)                                       Param #
RobertaForMaskedLM                                           --
├─RobertaModel: 1-1                                          --
│    └─RobertaEmbeddings: 2-1                                --
│    │    └─Embedding: 3-1                                   38,603,520
│    │    └─Embedding: 3-2                                   394,752
│    │    └─Embedding: 3-3                                   768
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─RobertaEncoder: 2-2                                   --
│    │    └─ModuleList: 3-6                                  85,054,464
├─RobertaLMHead: 1-2                                         --
│    └─Linear: 2-3                                           590,592
│    └─LayerNorm: 2-4                                        1,536
│    └─Linear: 2-5                                           38,65

Diese Model-Summary deutet noch auf ein weiteres Feature von Transformers hin: Viele Modelle wurden nicht nur für einen, sondern mehrere Aufgaben finetuned. In unserem Fall sehen wir, dass unser `RobertaForMaskedLM` Modell 2 Bestandteile hat: Das Kern-RoBERTa Modell (`RobertaModel`) und eine Gruppe an zusätzlichen Layern, die den rohen Output des `RobertaEncoder`s in die Logit-Gewichte umwandelt, die wir oben gesehen haben (`RobertaLMHead`). Im Falle von RoBERTa wurden auch *Heads* für andere Aufgabenstellungen spezifiziert, unter anderem für jenen Task, mit dem wir uns als nächstes beschäftigen werden: Named Entity Recognition (NER). Um das Modell mit einem Head für NER zu laden, könnten wir die `RobertaForTokenClassification`-Klasse verwenden. Da Transformers die Interfaces der Modelle für die gleiche Aufgabenstellung standardisiert, können wir stattdessen eine der sogenannten `Auto`-Klassen verwenden, die die Details zum Modell automatisch aus seiner Spezifikation lädt:

In [55]:
from transformers import AutoModelForTokenClassification

roberta_ner = AutoModelForTokenClassification.from_pretrained("roberta-base")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

Wie wir an der Warnung erkennen, wurde zwar das Format für einen NER Head für RoBERTa spezifiziert, der Checkpoint `roberta-base` enthält jedoch keine Weights für diesen Head, weshalb wir ihn erst für unseren spezifischen Einsatzzweck trainieren müssen.

## Named Entity Recognition (NER)



In [16]:
from datasets import load_dataset

bionlp2004 = load_dataset("tner/bionlp2004")

Found cached dataset bionlp2004 (/Users/jan/.cache/huggingface/datasets/tner___bionlp2004/bionlp2004/1.0.0/9f41d3f0270b773c2762dee333ae36c29331e2216114a57081f77639fdb5e904)


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
label2id = {
    "O": 0,
    "B-DNA": 1,
    "I-DNA": 2,
    "B-protein": 3,
    "I-protein": 4,
    "B-cell_type": 5,
    "I-cell_type": 6,
    "B-cell_line": 7,
    "I-cell_line": 8,
    "B-RNA": 9,
    "I-RNA": 10
}

id2label = { id: label for label, id in label2id.items()}

In [18]:
import numpy as np
import evaluate

seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base", add_prefix_space=True)

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    word_ids_batch = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        word_ids_batch.append(word_ids)

    tokenized_inputs["labels"] = labels
    tokenized_inputs["word_ids"] = word_ids_batch
    return tokenized_inputs

In [21]:
tokenized_bionlp2004 = bionlp2004.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /Users/jan/.cache/huggingface/datasets/tner___bionlp2004/bionlp2004/1.0.0/9f41d3f0270b773c2762dee333ae36c29331e2216114a57081f77639fdb5e904/cache-4c6521d889da19bb.arrow


Map:   0%|          | 0/1927 [00:00<?, ? examples/s]

Loading cached processed dataset at /Users/jan/.cache/huggingface/datasets/tner___bionlp2004/bionlp2004/1.0.0/9f41d3f0270b773c2762dee333ae36c29331e2216114a57081f77639fdb5e904/cache-b9b104ba286b51ab.arrow


In [7]:
from transformers import AutoModelForTokenClassification 

model = AutoModelForTokenClassification.from_pretrained(
    "roberta-large", num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [56]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType

peft_config_lora = LoraConfig(
    task_type=TaskType.TOKEN_CLS, inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1, bias="all"
)

peft_model_lora = get_peft_model(model, peft_config_lora).to("mps")
peft_model_lora.print_trainable_parameters()

trainable params: 1080342 || all params: 355119126 || trainable%: 0.30421960432511314


In [13]:
from transformers import TrainingArguments

lr = 1e-3
batch_size = 16
num_epochs = 10

training_args_lora = TrainingArguments(
    output_dir="roberta-large-lora-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_mps_device=True
)

In [14]:
from transformers import Trainer

trainer_lora = Trainer(
    model=peft_model_lora,
    args=training_args_lora,
    train_dataset=tokenized_bionlp2004["train"],
    eval_dataset=tokenized_bionlp2004["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer_lora.train()

/Users/jan/Git/ml-workshop/.venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10390 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [10]:
from peft import get_peft_model, PrefixTuningConfig, TaskType

peft_config_prefix = PrefixTuningConfig(task_type=TaskType.TOKEN_CLS, inference_mode=False, num_virtual_tokens=20)

peft_model_prefix = get_peft_model(model, peft_config_prefix).to("mps")
peft_model_prefix.print_trainable_parameters()

trainable params: 983040 || all params: 355315734 || trainable%: 0.2766666111104441


In [12]:
from transformers import TrainingArguments

lr = 1e-3
batch_size = 16
num_epochs = 10

training_args_prefix = TrainingArguments(
    output_dir="roberta-large-prefix-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_mps_device=True
)

In [15]:
from transformers import Trainer

trainer_prefix = Trainer(
    model=peft_model_prefix,
    args=training_args_prefix,
    train_dataset=tokenized_bionlp2004["train"],
    eval_dataset=tokenized_bionlp2004["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer_prefix.train()

/Users/jan/Git/ml-workshop/.venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10390 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.3354, 'learning_rate': 0.0009518768046198268, 'epoch': 0.48}


RuntimeError: MPS backend out of memory (MPS allocated: 4.53 GB, other allocations: 31.74 GB, max allowed: 36.27 GB). Tried to allocate 6.94 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [12]:
from transformers import Trainer
from transformers import TrainingArguments

lr = 1e-3
batch_size = 16
num_epochs = 10

full_training_args = TrainingArguments(
    output_dir="roberta-large-token-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_mps_device=True
)

full_trainer = Trainer(
    model=model,
    args=full_training_args,
    train_dataset=tokenized_bionlp2004["train"],
    eval_dataset=tokenized_bionlp2004["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

full_trainer.train()

/Users/jan/Git/ml-workshop/.venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10390 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [52]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
from spacy import displacy

detokenizer = TreebankWordDetokenizer()
sample = tokenized_bionlp2004["train"][2]


def convert_to_spacy_format(sample):
    text = detokenizer.detokenize(sample["tokens"])
    detected_entities = []
    prev_word_idx = None
    running_label = None
    running_entity = []
    search_start = 0

    def complete_detected_entity(new_label):
        nonlocal detected_entities, running_label, running_entity, search_start

        if running_label is not None:
            phrase = detokenizer.detokenize(running_entity)
            phrase_start = text.find(phrase, search_start)
            search_start = phrase_start + len(phrase)
            detected_entities.append({
                "label": id2label[running_label],
                "start": phrase_start,
                "end": phrase_start + len(phrase)
            })
        running_label = new_label
        running_entity = []

    for i, word_idx in enumerate(sample["word_ids"]):
        label = sample["labels"][i]
        if prev_word_idx == word_idx:
            continue
        
        if not (label == -100 or label == 0):
            if running_label != label:
                complete_detected_entity(new_label=label)
            running_entity.append(sample["tokens"][word_idx])
        else: 
            complete_detected_entity(None)

    complete_detected_entity(None)

    return {
        "ents": detected_entities,
        "text": text
    }

displacy.render(convert_to_spacy_format(sample), style="ent", manual=True)

In [40]:
from transformers.adapters import AdapterTrainer, AutoAdapterModel
from transformers import AutoConfig, AdapterConfig, RobertaAdapterModel, AutoModelForTokenClassification

# adapter_config = AdapterConfig.load(
#     None,
#     leave_out=[11]
# )

model_adapter_config = AutoConfig.from_pretrained("roberta-large", num_labels=len(id2label), id2label=id2label, label2id=label2id)
model_adapter = RobertaAdapterModel.from_pretrained("roberta-large", config=model_adapter_config)

# model_adapter.add_adapter("bio_ner")
# model_adapter.train_adapter("bio_ner")

# training_args_adapter = TrainingArguments(
#     learning_rate=1e-4,
#     num_train_epochs=6,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     logging_steps=200,
#     output_dir="./roberta-large-adapter-token-classification",
#     overwrite_output_dir=True,
#     # The next line is important to ensure the dataset labels are properly passed to the model
#     remove_unused_columns=False,
# )

# trainer = AdapterTrainer(
#     model=model,
#     args=training_args_adapter,
#     train_dataset=tokenized_bionlp2004["train"],
#     eval_dataset=tokenized_bionlp2004["validation"],
#     compute_metrics=compute_metrics,
# )

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaAdapterModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaAdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaAdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
model_adapter.add_adapter("bio_ner")

AttributeError: 'RobertaModel' object has no attribute 'add_adapter'